In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
sns.set()
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [75]:
auth.authenticate_user()
print('Authenticated')


Authenticated


In [0]:
project_id='eicudata'

# Extracting features from the eICU dataset for Mortality and los prediction

In [0]:
%%bigquery --project $project_id rawdf
select b.patientunitstayid,unitvisitnumber, gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight, temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,(unitdischargeoffset-hospitaladmitoffset)/60/24 as ICUstay
from `physionet-data.eicu_crd.apacheapsvar` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
order by uniquepid 

In [78]:
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,224606,1,Female,76,Caucasian,"GI perforation/rupture, surgery for",160.0,NaN,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250
1,204602,1,Female,29,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389
2,157016,1,Female,23,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417
3,211144,1,Female,67,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056
4,221005,1,Female,68,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171172,3351785,1,Male,74,Caucasian,Head only trauma,182.9,75.7,36.8,49.0,50,-1.0,-1.0,-1.00,-1.000,Alive,Alive,1.790278
171173,3340321,1,Male,44,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583
171174,3345874,1,Male,55,African American,Rhythm disturbance (conduction defect),190.5,165.9,32.2,37.0,90,254.0,-1.0,2.30,-1.000,Alive,Alive,1.968056
171175,3352628,1,Male,60,African American,"Apnea-sleep; surgery for (i.e., UPPP - uvulopa...",170.1,120.2,36.3,37.0,114,102.0,50.0,1.00,7.363,Alive,Alive,1.481250


## Finding number of missing or null values -

In [79]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171177 entries, 0 to 171176
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171177 non-null  object 
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          167853 non-null  float64
 7   admissionweight          164555 non-null  float64
 8   temperature              171177 non-null  float64
 9   respiratoryrate          171177 non-null  float64
 10  heartrate                171177 non-null  int64  
 11  glucose                  171177 non-null  float64
 12  fio2                     171177 non-null  float64
 13  creatinine               171177 non-null  float64
 14  ph  

## Statistics of the features for preprocessing - 

In [80]:
rawdf.describe(include='all')

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
count,1.711770e+05,171177.000000,171177,171177,171177,171177,167853.000000,164555.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177,171177,171177.000000
unique,NaN,NaN,5,90,7,393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,NaN
top,NaN,NaN,Male,> 89,Caucasian,"Sepsis, pulmonary",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,NaN
freq,NaN,NaN,92468,6084,131319,8427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153532,161148,NaN
mean,1.747701e+06,1.151048,NaN,NaN,NaN,NaN,169.266201,83.850422,33.728719,24.784735,98.667993,141.184455,12.794373,0.999311,0.906831,NaN,NaN,4.254612
std,9.988850e+05,0.508644,NaN,NaN,NaN,NaN,13.845103,27.082294,9.725657,15.301895,33.263814,109.723423,28.329034,1.852455,3.506569,NaN,NaN,6.893154
min,1.411680e+05,1.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,-0.210417
25%,9.555340e+05,1.000000,NaN,NaN,NaN,NaN,162.500000,66.000000,36.100000,10.000000,84.000000,85.000000,-1.000000,0.470000,-1.000000,NaN,NaN,1.209028
50%,1.670376e+06,1.000000,NaN,NaN,NaN,NaN,170.000000,79.900000,36.400000,27.000000,104.000000,115.000000,-1.000000,0.810000,-1.000000,NaN,NaN,2.236111
75%,2.743419e+06,1.000000,NaN,NaN,NaN,NaN,177.800000,96.800000,36.700000,35.000000,120.000000,187.000000,-1.000000,1.360000,-1.000000,NaN,NaN,4.772222


## Impute missing values for Admission height and Admission weight 

In [0]:
rawdf['admissionheight']=rawdf.admissionheight.fillna(rawdf.admissionheight.median())
rawdf['admissionweight']=rawdf.admissionweight.fillna(rawdf.admissionweight.median())

In [82]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171177 entries, 0 to 171176
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171177 non-null  object 
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          171177 non-null  float64
 7   admissionweight          171177 non-null  float64
 8   temperature              171177 non-null  float64
 9   respiratoryrate          171177 non-null  float64
 10  heartrate                171177 non-null  int64  
 11  glucose                  171177 non-null  float64
 12  fio2                     171177 non-null  float64
 13  creatinine               171177 non-null  float64
 14  ph  

## Convert age values to numeric

In [0]:
rawdf['age'].replace({'> 89': '90'}, inplace=True)
rawdf['age']=rawdf['age'].apply(pd.to_numeric)

In [84]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171177 entries, 0 to 171176
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patientunitstayid        171177 non-null  int64  
 1   unitvisitnumber          171177 non-null  int64  
 2   gender                   171177 non-null  object 
 3   age                      171109 non-null  float64
 4   ethnicity                171177 non-null  object 
 5   apacheadmissiondx        171177 non-null  object 
 6   admissionheight          171177 non-null  float64
 7   admissionweight          171177 non-null  float64
 8   temperature              171177 non-null  float64
 9   respiratoryrate          171177 non-null  float64
 10  heartrate                171177 non-null  int64  
 11  glucose                  171177 non-null  float64
 12  fio2                     171177 non-null  float64
 13  creatinine               171177 non-null  float64
 14  ph  

## Age has some null values. Removing unknown/other category in gender and ethnicity. : 

In [0]:
missing = rawdf[(rawdf['gender'] == '') |(rawdf['gender'] == 'Other')|(rawdf['gender'] == 'Unknown')| (rawdf['ethnicity'] == '')|(rawdf['ethnicity'] == 'Other/Unknown')|(rawdf['unitdischargestatus']=='')|(rawdf['hospitaldischargestatus']=='')].index
rawdf = rawdf.drop(index=missing)
rawdf = rawdf.dropna()

## Checking if we have no more missing or ambiguous values for features

In [86]:
rawdf.isnull().sum()

patientunitstayid          0
unitvisitnumber            0
gender                     0
age                        0
ethnicity                  0
apacheadmissiondx          0
admissionheight            0
admissionweight            0
temperature                0
respiratoryrate            0
heartrate                  0
glucose                    0
fio2                       0
creatinine                 0
ph                         0
hospitaldischargestatus    0
unitdischargestatus        0
ICUstay                    0
dtype: int64

## Exclusion criteria for mortality prediction
            age <= 18 and age >= 89
            more than 1 ICU stay
            los < 48h

In [87]:
excl = rawdf[(rawdf['age']<= 18) | (rawdf['age']>=89) |(rawdf['unitvisitnumber']>1)| (rawdf['ICUstay']<2)].index
rawdf = rawdf.drop(index=excl)
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250
1,204602,1,Female,29.0,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389
2,157016,1,Female,23.0,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056
4,221005,1,Female,68.0,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91,82.0,100.0,1.19,7.356,Alive,Alive,6.891667
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",193.0,109.1,35.8,5.0,91,181.0,100.0,0.60,7.315,Alive,Alive,3.617361
171171,3329614,1,Male,71.0,African American,"Hemorrhage/hematoma, intracranial",188.0,90.7,36.7,50.0,109,97.0,-1.0,2.52,-1.000,Expired,Alive,3.704167
171173,3340321,1,Male,44.0,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583


In [88]:
rawdf.describe(include='all')

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay
count,7.025000e+04,70250.0,70250,70250.000000,70250,70250,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250,70250,70250.000000
unique,NaN,NaN,2,NaN,5,390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN
top,NaN,NaN,Male,NaN,Caucasian,"Sepsis, pulmonary",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,NaN
freq,NaN,NaN,38425,NaN,56581,4055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62784,65829,NaN
mean,1.773296e+06,1.0,NaN,63.398904,NaN,NaN,169.620476,85.245969,34.494745,25.655966,103.247815,151.463442,18.918431,1.215924,1.708280,NaN,NaN,5.964793
std,1.011975e+06,0.0,NaN,15.458700,NaN,NaN,12.875890,27.444035,8.295363,15.203496,31.329025,107.663290,32.340339,1.910404,3.912488,NaN,NaN,7.515822
min,1.411680e+05,1.0,NaN,19.000000,NaN,NaN,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,2.000000
25%,9.728780e+05,1.0,NaN,54.000000,NaN,NaN,162.600000,67.600000,36.000000,11.000000,91.000000,91.000000,-1.000000,0.570000,-1.000000,NaN,NaN,2.770139
50%,1.646298e+06,1.0,NaN,65.000000,NaN,NaN,170.100000,80.700000,36.400000,28.000000,107.000000,129.000000,-1.000000,0.900000,-1.000000,NaN,NaN,3.998611
75%,2.804245e+06,1.0,NaN,75.000000,NaN,NaN,177.800000,98.200000,36.700000,36.000000,123.000000,195.000000,40.000000,1.540000,7.300000,NaN,NaN,6.831250


## Encoding the categorical values

In [0]:
lb_make = LabelEncoder()

In [90]:
rawdf['ethnicity_code'] = lb_make.fit_transform(rawdf['ethnicity'])
rawdf['diag_code'] = lb_make.fit_transform(rawdf['apacheadmissiondx'])
rawdf['gender_code'] = rawdf['gender'].map({"Male":0,"Female":1})
rawdf['hosp_alive'] = rawdf['hospitaldischargestatus'].map({"Expired":0,"Alive":1})
rawdf['icu_alive'] = rawdf['unitdischargestatus'].map({"Expired":0,"Alive":1})
rawdf

,patientunitstayid,unitvisitnumber,gender,age,ethnicity,apacheadmissiondx,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,hospitaldischargestatus,unitdischargestatus,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
0,224606,1,Female,76.0,Caucasian,"GI perforation/rupture, surgery for",160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,Alive,Alive,3.031250,2,172,1,1,1
1,204602,1,Female,29.0,Caucasian,"Cardiovascular medical, other",162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,Alive,Alive,4.201389,2,92,1,1,1
2,157016,1,Female,23.0,Caucasian,"GI medical, other",162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,Alive,Alive,5.210417,2,168,1,1,1
3,211144,1,Female,67.0,Caucasian,Aortic valve replacement (isolated),160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,Alive,Alive,3.893056,2,34,1,1,1
4,221005,1,Female,68.0,Caucasian,"Complications of previous open-heart surgery, ...",162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,Alive,Alive,2.803472,2,119,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171169,3351729,1,Male,64.0,African American,"CABG alone, coronary artery bypass grafting",182.9,112.5,36.1,41.0,91,82.0,100.0,1.19,7.356,Alive,Alive,6.891667,0,60,0,1,1
171170,3340416,1,Male,64.0,Caucasian,"CABG alone, coronary artery bypass grafting",193.0,109.1,35.8,5.0,91,181.0,100.0,0.60,7.315,Alive,Alive,3.617361,2,60,0,1,1
171171,3329614,1,Male,71.0,African American,"Hemorrhage/hematoma, intracranial",188.0,90.7,36.7,50.0,109,97.0,-1.0,2.52,-1.000,Expired,Alive,3.704167,0,209,0,0,1
171173,3340321,1,Male,44.0,Caucasian,"Infarction, acute myocardial (MI)",185.4,130.6,36.7,45.0,102,220.0,-1.0,0.91,-1.000,Alive,Alive,2.689583,2,229,0,1,1


## Handling outliers
### There are outliers in columns : admissionheight,admissionweight

In [91]:
rawdf.describe()

,patientunitstayid,unitvisitnumber,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,diag_code,gender_code,hosp_alive,icu_alive
count,7.025000e+04,70250.0,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000,70250.000000
mean,1.773296e+06,1.0,63.398904,169.620476,85.245969,34.494745,25.655966,103.247815,151.463442,18.918431,1.215924,1.708280,5.964793,1.778320,192.558391,0.453025,0.893722,0.937068
std,1.011975e+06,0.0,15.458700,12.875890,27.444035,8.295363,15.203496,31.329025,107.663290,32.340339,1.910404,3.912488,7.515822,0.742443,116.400232,0.497792,0.308195,0.242843
min,1.411680e+05,1.0,19.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.728780e+05,1.0,54.000000,162.600000,67.600000,36.000000,11.000000,91.000000,91.000000,-1.000000,0.570000,-1.000000,2.770139,2.000000,73.000000,0.000000,1.000000,1.000000
50%,1.646298e+06,1.0,65.000000,170.100000,80.700000,36.400000,28.000000,107.000000,129.000000,-1.000000,0.900000,-1.000000,3.998611,2.000000,189.000000,0.000000,1.000000,1.000000
75%,2.804245e+06,1.0,75.000000,177.800000,98.200000,36.700000,36.000000,123.000000,195.000000,40.000000,1.540000,7.300000,6.831250,2.000000,322.000000,1.000000,1.000000,1.000000
max,3.353251e+06,1.0,88.000000,612.600000,953.000000,42.300000,60.000000,220.000000,1791.000000,100.000000,24.600000,7.810000,749.036111,4.000000,389.000000,1.000000,1.000000,1.000000


In [0]:
rawdf['admissionheight'] = np.where(rawdf['admissionheight']<rawdf['admissionheight'].quantile(0.10),rawdf['admissionheight'].quantile(0.10),rawdf['admissionheight'])
rawdf['admissionheight'] = np.where(rawdf['admissionheight']>rawdf['admissionheight'].quantile(0.90),rawdf['admissionheight'].quantile(0.90),rawdf['admissionheight'])
rawdf['admissionweight'] = np.where(rawdf['admissionweight']<rawdf['admissionweight'].quantile(0.10),rawdf['admissionweight'].quantile(0.10),rawdf['admissionweight'])
rawdf['admissionweight'] = np.where(rawdf['admissionweight']>rawdf['admissionweight'].quantile(0.90),rawdf['admissionweight'].quantile(0.90),rawdf['admissionweight'])
rawdf['temperature'] = np.where(rawdf['temperature']<rawdf['temperature'].quantile(0.10),rawdf['temperature'].quantile(0.10),rawdf['temperature'])
rawdf['respiratoryrate'] = np.where(rawdf['respiratoryrate']<rawdf['respiratoryrate'].quantile(0.10),rawdf['respiratoryrate'].quantile(0.10),rawdf['respiratoryrate'])
rawdf['heartrate'] = np.where(rawdf['heartrate']<rawdf['heartrate'].quantile(0.10),rawdf['heartrate'].quantile(0.10),rawdf['heartrate'])
rawdf['glucose'] = np.where(rawdf['glucose']<rawdf['glucose'].quantile(0.10),rawdf['glucose'].quantile(0.10),rawdf['glucose'])
rawdf['glucose'] = np.where(rawdf['glucose']>rawdf['glucose'].quantile(0.90),rawdf['glucose'].quantile(0.90),rawdf['glucose'])

In [94]:
print(rawdf['temperature'].quantile(0.10))
print(rawdf['respiratoryrate'].quantile(0.10))
print(rawdf['heartrate'].quantile(0.10))
print(rawdf['glucose'].quantile(0.10))
print(rawdf['fio2'].quantile(0.10))
print(rawdf['creatinine'].quantile(0.15))
print(rawdf['ph'].quantile(0.15))

35.1
7.0
55.0
55.0
-1.0
-1.0
-1.0


In [103]:
#index = rawdf[(rawdf['fio2']==-1)|(rawdf['creatinine']==-1)|(rawdf['ph']==-1)].index
index = rawdf[(rawdf['ph']==-1)].index
#rawdf.drop(index=index,inplace=True)
#rawdf
index

Int64Index([     1,      2,      6,      7,      8,     10,     13,     15,
                25,     26,
            ...
            171138, 171145, 171149, 171151, 171153, 171157, 171164, 171166,
            171171, 171173],
           dtype='int64', length=47490)

In [95]:
rawdf.describe()
rawdf.skew()

patientunitstayid     0.039418
unitvisitnumber       0.000000
age                  -0.654384
admissionheight      -0.083459
admissionweight       0.336061
temperature           1.296816
respiratoryrate       0.280437
heartrate            -0.100728
glucose               0.491053
fio2                  1.411862
creatinine            2.875076
ph                    0.752830
ICUstay              32.567965
ethnicity_code       -1.410416
diag_code             0.006570
gender_code           0.188739
hosp_alive           -2.555096
icu_alive            -3.599692
dtype: float64

In [58]:
rawdf.drop(['patientunitstayid','apacheadmissiondx','ethnicity','gender','unitdischargestatus','hospitaldischargestatus'],axis=1)

,unitvisitnumber,age,admissionheight,admissionweight,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph,ICUstay,ethnicity_code,gender_code,is_alive,hosp_alive,icu_alive,diag_code
0,1,76.0,160.0,79.9,36.7,38.0,119,81.0,100.0,1.10,7.280,3.031250,2,1,1,1,1,172
1,1,29.0,162.6,88.5,36.5,55.0,138,82.0,-1.0,3.21,-1.000,4.201389,2,1,1,1,1,92
2,1,23.0,162.6,63.5,36.4,45.0,115,82.0,-1.0,0.61,-1.000,5.210417,2,1,1,1,1,168
3,1,67.0,160.0,86.2,36.8,33.0,104,85.0,50.0,0.80,7.380,3.893056,2,1,1,1,1,34
4,1,68.0,162.6,88.0,36.4,43.0,87,37.0,32.0,-1.00,7.340,2.803472,2,1,1,1,1,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171169,1,64.0,182.9,112.5,36.1,41.0,91,82.0,100.0,1.19,7.356,6.891667,0,0,1,1,1,60
171170,1,64.0,182.9,109.1,35.8,5.0,91,181.0,100.0,0.60,7.315,3.617361,2,0,1,1,1,60
171171,1,71.0,182.9,90.7,36.7,50.0,109,97.0,-1.0,2.52,-1.000,3.704167,0,0,1,0,1,209
171173,1,44.0,182.9,117.9,36.7,45.0,102,220.0,-1.0,0.91,-1.000,2.689583,2,0,1,1,1,229
